<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Tablets" data-toc-modified-id="Tablets-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Tablets</a></span></li><li><span><a href="#Faces" data-toc-modified-id="Faces-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Faces</a></span></li><li><span><a href="#Columns" data-toc-modified-id="Columns-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Columns</a></span></li><li><span><a href="#Lines" data-toc-modified-id="Lines-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Lines</a></span><ul class="toc-item"><li><span><a href="#Line-numbers" data-toc-modified-id="Line-numbers-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Line numbers</a></span></li><li><span><a href="#Cross-references" data-toc-modified-id="Cross-references-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Cross references</a></span></li></ul></li><li><span><a href="#Comments" data-toc-modified-id="Comments-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Comments</a></span><ul class="toc-item"><li><span><a href="#Object-descriptions" data-toc-modified-id="Object-descriptions-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Object descriptions</a></span></li><li><span><a href="#Rulings" data-toc-modified-id="Rulings-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Rulings</a></span></li><li><span><a href="#Metadata" data-toc-modified-id="Metadata-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Metadata</a></span></li></ul></li><li><span><a href="#Graphemes" data-toc-modified-id="Graphemes-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Graphemes</a></span><ul class="toc-item"><li><span><a href="#Primes" data-toc-modified-id="Primes-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Primes</a></span></li><li><span><a href="#Variants-and-modifiers" data-toc-modified-id="Variants-and-modifiers-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Variants and modifiers</a></span><ul class="toc-item"><li><span><a href="#Order" data-toc-modified-id="Order-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>Order</a></span></li><li><span><a href="#Uppercase" data-toc-modified-id="Uppercase-6.2.2"><span class="toc-item-num">6.2.2&nbsp;&nbsp;</span>Uppercase</a></span></li><li><span><a href="#Stray-modifier" data-toc-modified-id="Stray-modifier-6.2.3"><span class="toc-item-num">6.2.3&nbsp;&nbsp;</span>Stray modifier</a></span></li></ul></li><li><span><a href="#Tweaks" data-toc-modified-id="Tweaks-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Tweaks</a></span></li><li><span><a href="#Flags" data-toc-modified-id="Flags-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Flags</a></span><ul class="toc-item"><li><span><a href="#A-bit-of-research" data-toc-modified-id="A-bit-of-research-6.4.1"><span class="toc-item-num">6.4.1&nbsp;&nbsp;</span>A bit of research</a></span></li></ul></li><li><span><a href="#All-signs" data-toc-modified-id="All-signs-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>All signs</a></span></li></ul></li><li><span><a href="#Quads" data-toc-modified-id="Quads-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Quads</a></span><ul class="toc-item"><li><span><a href="#Outer-complex-quads" data-toc-modified-id="Outer-complex-quads-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Outer complex quads</a></span></li><li><span><a href="#Variants:-inside-or-outside?" data-toc-modified-id="Variants:-inside-or-outside?-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Variants: inside or outside?</a></span></li><li><span><a href="#Variants:-extra-level-of-brackets?" data-toc-modified-id="Variants:-extra-level-of-brackets?-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Variants: extra level of brackets?</a></span></li><li><span><a href="#More-bracket-issues" data-toc-modified-id="More-bracket-issues-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>More bracket issues</a></span></li><li><span><a href="#The-most-complex-quads" data-toc-modified-id="The-most-complex-quads-7.5"><span class="toc-item-num">7.5&nbsp;&nbsp;</span>The most complex quads</a></span></li></ul></li><li><span><a href="#Clusters" data-toc-modified-id="Clusters-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Clusters</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></div>

# Checks
Various checks on the correctness of the transformation from ascii transcriptions to a text-fabric data set.

The
[diagnostics](https://github.com/Dans-labs/Nino-cunei/blob/master/reports/diagnostics.tsv)
of the transformation contains valueable issues that may be used to correct mistakes in the sources.
Or, equally likely, they correspond to misunderstandings on my (Dirk's) part of the model
that underlies the transcriptions.

We will perform *grep* commands on the source files, and we will traverse node in Text-Fabric and collect information.

Then we compare these sets of information.

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import sys, os, collections, re
from glob import glob
from tf.fabric import Fabric
from tf.cunei import Cunei
from utils import Compare

In [8]:
REPO = '~/github/Dans-labs/Nino-cunei'
SOURCE = 'uruk'
VERSION = '0.1'
CORPUS = f'{REPO}/tf/{SOURCE}/{VERSION}'
SOURCE_DIR = os.path.expanduser(f'{REPO}/sources/cdli')
TEMP_DIR = os.path.expanduser(f'{REPO}/_temp')

In [9]:
TF = Fabric(locations=[CORPUS], modules=[''], silent=False )

This is Text-Fabric 3.2.1
Api reference : https://github.com/Dans-labs/text-fabric/wiki/Api
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

33 features found and 0 ignored


In [10]:
api = TF.load('''
    grapheme prime repeat
    variant variantOuter
    modifier modifierInner modifierFirst
    damage uncertain remarkable written
    kind
    period name type identifier catalogId
    number fullNumber origNumber badNumbering
    crossref text
    srcLn srcLnNum
    op sub comments
''')
api.makeAvailableIn(globals())
CUNEI = Cunei(api)
COMP = Compare(api, SOURCE_DIR, TEMP_DIR)

  0.00s loading features ...
   |     0.00s B catalogId            from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.02s B fullNumber           from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.01s B number               from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.06s B grapheme             from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.04s B srcLn                from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.02s B srcLnNum             from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.00s B prime                from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.01s B repeat               from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.01s B variant              from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.00s B variantOuter         from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.00s B modi

## Tablets
We check whether we have the same sequence of tablet numbers.
In TF, the tablet number is stored in the feature `catalogId`.

Note that we also check on the order of the tablets.

In [11]:
def tfTablets():
    tablets = []
    for t in F.otype.s('tablet'):
        (tablet, column, line) = T.sectionFromNode(t)
        tablets.append((F.period.v(t), tablet, F.srcLnNum.v(t), F.catalogId.v(t)))
    return tablets

def grepTablets(gen):
    tablets = []
    prevTablet = None
    for (period, tablet, ln, line, skip) in gen:
        if skip:
            print(f'GREP: skipping duplicate tablet "{tablet}"')
            continue
        if tablet != prevTablet:
            tablets.append((period, tablet, ln, tablet))
        prevTablet = tablet
    return tablets

In [12]:
COMP.checkSanity(
    ('tablet',),
    grepTablets,
    tfTablets,
)

GREP: skipping duplicate tablet "P002176"
GREP: skipping duplicate tablet "P252175"
HEAD : period ◆ tablet ◆ ln ◆ tablet
IDENTICAL: all 6396 items
=    : uruk-iii ◆ P006427 ◆ 1 ◆ P006427
=    : uruk-iii ◆ P006428 ◆ 11 ◆ P006428
=    : uruk-iii ◆ P448701 ◆ 36 ◆ P448701
=    : uruk-iii ◆ P448702 ◆ 50 ◆ P448702
=    : uruk-iii ◆ P448703 ◆ 71 ◆ P448703
=    : uruk-iii ◆ P471695 ◆ 87 ◆ P471695
=    : uruk-iii ◆ P482082 ◆ 114 ◆ P482082
=    : uruk-iii ◆ P482083 ◆ 127 ◆ P482083
=    : uruk-iii ◆ P499393 ◆ 147 ◆ P499393
=    : uruk-iii ◆ P504412 ◆ 166 ◆ P504412
=    : uruk-iii ◆ P504413 ◆ 189 ◆ P504413
=    : uruk-iii ◆ P006438 ◆ 199 ◆ P006438
=    : uruk-iii ◆ P000014 ◆ 220 ◆ P000014
=    : uruk-iii ◆ P000456 ◆ 297 ◆ P000456
=    : uruk-iii ◆ P002718 ◆ 326 ◆ P002718
=    : uruk-iii ◆ P000021 ◆ 341 ◆ P000021
=    : uruk-iii ◆ P000023 ◆ 374 ◆ P000023
=    : uruk-iii ◆ P000025 ◆ 403 ◆ P000025
=    : uruk-iii ◆ P000167 ◆ 500 ◆ P000167
=    : uruk-iii ◆ P000453 ◆ 531 ◆ P000453
=     and 6376 more


## Faces

We check whether we see the same faces with GREP and TF.

Note that in TF we have inserted missing faces `@noface`.
We leave them out again in the comparison.

In [13]:
FACES = set(
    '''
    obverse
    reverse
    top
    bottom
    left
    seal
    surface
    edge
'''.strip().split()
)

NOFACE = 'noface'

faceDetect = '''
(?:
    ^@([a-z]+)
)
'''

In [14]:
def tfFaces():
    faces = []
    for tablet in F.otype.s('tablet'):
        tabletName = F.catalogId.v(tablet)
        period = F.period.v(tablet)
        for face in L.d(tablet, otype='face'):
            tp = F.type.v(face)
            it = F.identifier.v(face) or None
            ln = F.srcLnNum.v(face)
            itStr = '' if it is None else f' {it}'
            if tp != 'noface':
                faces.append((period, tabletName, ln, f'@{tp}{itStr}'))
    return faces

In [15]:
def grepFaces(gen):
    faces = []
    faceRe = re.compile(faceDetect, re.X)
    for (period, tablet, ln, line, skip) in gen:
        if skip:
            continue
        match = faceRe.match(line)
        if match:
            face = match.group(1)
            if face in FACES:
                faces.append((period, tablet, ln, line.strip()))
    return faces

In [16]:
COMP.checkSanity(
    ('face',),
    grepFaces,
    tfFaces,
)

HEAD : period ◆ tablet ◆ ln ◆ face
IDENTICAL: all 9441 items
=    : uruk-iii ◆ P006427 ◆ 4 ◆ @obverse
=    : uruk-iii ◆ P006428 ◆ 14 ◆ @obverse
=    : uruk-iii ◆ P448701 ◆ 39 ◆ @obverse
=    : uruk-iii ◆ P448701 ◆ 46 ◆ @reverse
=    : uruk-iii ◆ P448702 ◆ 53 ◆ @obverse
=    : uruk-iii ◆ P448702 ◆ 67 ◆ @reverse
=    : uruk-iii ◆ P448703 ◆ 74 ◆ @obverse
=    : uruk-iii ◆ P448703 ◆ 83 ◆ @reverse
=    : uruk-iii ◆ P471695 ◆ 90 ◆ @obverse
=    : uruk-iii ◆ P471695 ◆ 109 ◆ @reverse
=    : uruk-iii ◆ P482082 ◆ 117 ◆ @obverse
=    : uruk-iii ◆ P482082 ◆ 123 ◆ @reverse
=    : uruk-iii ◆ P482083 ◆ 130 ◆ @obverse
=    : uruk-iii ◆ P482083 ◆ 143 ◆ @reverse
=    : uruk-iii ◆ P499393 ◆ 150 ◆ @obverse
=    : uruk-iii ◆ P499393 ◆ 162 ◆ @reverse
=    : uruk-iii ◆ P504412 ◆ 169 ◆ @obverse
=    : uruk-iii ◆ P504412 ◆ 185 ◆ @reverse
=    : uruk-iii ◆ P504413 ◆ 192 ◆ @obverse
=    : uruk-iii ◆ P504413 ◆ 195 ◆ @reverse
=     and 9421 more
Number of results: TF 9441; GREP 9441


## Columns

We check whether we see the same columns with GREP and TF.

Note that in TF we have inserted missing columns as `@column 0`.
We leave them out again in the comparison.

In [17]:
def tfColumns():
    columns = []
    for tablet in F.otype.s('tablet'):
        tabletName = F.catalogId.v(tablet)
        period = F.period.v(tablet)
        for face in L.d(tablet, otype='face'):
            tp = F.type.v(face)
            for column in L.d(face, otype='column'):
                number = F.number.v(column)
                prime = F.prime.v(column)
                ln = F.srcLnNum.v(column)
                primeStr = "'" if prime else ''
                if number != '0':
                    columns.append((period, tabletName, ln, tp, f'@column {number}{primeStr}'))
    return columns

In [18]:
columnDetect = '''
(?:
    ^@col
)
'''

columnCorrectPat = '''
(?:
    ^@([a-z]+)
    (\s*)
    (\S*)
)
'''

In [19]:
def grepColumns(gen):
    columnRe = re.compile(columnDetect, re.X)
    correctRe = re.compile(columnCorrectPat, re.X)
    faceRe = re.compile(faceDetect, re.X)

    columns = []
    curFace = NOFACE
    prevTablet = None
    for (period, tablet, ln, line, skip) in gen:
        if skip:
            continue
        if tablet != prevTablet:
            curFace = NOFACE
        prevTablet = tablet

        match = faceRe.match(line)

        if match:
            face = match.group(1)
            if face in FACES:
                curFace = face

        if columnRe.match(line):
            if not line.startswith('@column '):
                match = correctRe.match(line)
                if match:
                    colSpec = match.group(1)
                    sep = match.group(2)
                    colNum = match.group(3)
                    line = f'@column {colNum}'
                    print(f'GREP: corrected "{colSpec}{sep}{colNum}" => "{line}"')
                else:
                    print(f'GREP: found "{line}"')
                
            columns.append((period, tablet, ln, curFace, line.strip()))
    return columns

In [20]:
COMP.checkSanity(
    ('face', 'column'),
    grepColumns,
    tfColumns,
)

GREP: corrected "columm 4" => "@column 4"
GREP: corrected "column3" => "@column 3"
HEAD : period ◆ tablet ◆ ln ◆ face ◆ column
IDENTICAL: all 13123 items
=    : uruk-iii ◆ P006427 ◆ 5 ◆ obverse ◆ @column 1
=    : uruk-iii ◆ P006427 ◆ 7 ◆ obverse ◆ @column 2
=    : uruk-iii ◆ P006428 ◆ 15 ◆ obverse ◆ @column 1
=    : uruk-iii ◆ P006428 ◆ 18 ◆ obverse ◆ @column 2
=    : uruk-iii ◆ P006428 ◆ 21 ◆ obverse ◆ @column 3
=    : uruk-iii ◆ P006428 ◆ 29 ◆ obverse ◆ @column 4
=    : uruk-iii ◆ P006428 ◆ 32 ◆ obverse ◆ @column 5
=    : uruk-iii ◆ P448701 ◆ 40 ◆ obverse ◆ @column 1
=    : uruk-iii ◆ P448701 ◆ 43 ◆ obverse ◆ @column 2
=    : uruk-iii ◆ P448702 ◆ 54 ◆ obverse ◆ @column 1
=    : uruk-iii ◆ P448702 ◆ 60 ◆ obverse ◆ @column 2
=    : uruk-iii ◆ P448702 ◆ 64 ◆ obverse ◆ @column 3
=    : uruk-iii ◆ P448703 ◆ 75 ◆ obverse ◆ @column 1
=    : uruk-iii ◆ P448703 ◆ 81 ◆ obverse ◆ @column 2
=    : uruk-iii ◆ P471695 ◆ 91 ◆ obverse ◆ @column 1
=    : uruk-iii ◆ P471695 ◆ 104 ◆ obverse ◆ @column 2

## Lines

We check whether we see the same line numbers with GREP and TF.
We also check the cross-references on lines.

The text material on lines will be checked below, systematically.

### Line numbers

During the conversion to TF we have 
detected bad numberings in some columns
and stored that fact in the `badNumbering` feature.

One way to look at them is in the raw TF file
[badNumbering.tf](https://github.com/Dans-labs/Nino-cunei/blob/master/tf/uruk/0.1/badNumbering.tf).

There you see case nodes with values `1` (duplicate numbers) or `2` (wrong order).

Here is an overview of the cases:

In [21]:
for (val, amount) in F.badNumbering.freqList():
    print(f'{amount:>3} x {val}')

 26 x 2
  3 x 1


For full detail, see the
[diagnostics](https://github.com/Dans-labs/Nino-cunei/blob/master/reports/diagnostics.tsv).

Note that the numbered lines in the transcriptions do not correspond to the TF node type `line`,
but to `case`. 
Because these lines are filled with material of the smallest cases (those that do not have
sub-cases).

In TF these are the cases that have the feature `fullNumber`.

In TF we have removed the dots from numbers, but kept them otherwise unchanged.
In order to make the comparison, we also remove the dots after grepping numbers from the source.

In [22]:
def tfLines():
    cases = []
    for tablet in F.otype.s('tablet'):
        tabletName = F.catalogId.v(tablet)
        period = F.period.v(tablet)
        for case in L.d(tablet, otype='case'):
            fullNumber = F.fullNumber.v(case)
            if fullNumber is None:
                continue
            ln = F.srcLnNum.v(case)
            origNumber = F.origNumber.v(case)
            theNumber = fullNumber if origNumber is None else origNumber 
            cases.append((period, tabletName, ln, f'{theNumber}'))
    return cases

In [23]:
lineNumPat = '''
(?:
    ^
    (
            (?:[a-zA-Z0-9.\'-]+)
        |
            (?=[|\[])
    )
)
'''
lineNumDetect = '''
(?:
    ^
    (?:
            (?:[a-zA-Z0-9.\'-]+\s+)
        |
            (?=[|\[])
    )
)
'''

In [24]:
def tweakLine(line):
    if line.startswith('1.1('):
        print(f'GREP: "1.1(" => "1. 1("')
        line = line.replace('1.1(', '1. 1(', 1)
    return line

In [25]:
def grepLines(gen):
    cases = []
    lineNumRe = re.compile(lineNumPat, re.X)
    for (period, tablet, ln, line, skip) in gen:
        if skip:
            continue

        line = tweakLine(line)
        match = lineNumRe.match(line)
        if match:
            caseNum = match.group(1)
            caseNumClean = caseNum.replace('.', '').strip()
            cases.append((period, tablet, ln, caseNumClean))
    return cases

In [26]:
COMP.checkSanity(
    ('lineNum',),
    grepLines,
    tfLines,
)

GREP: "1.1(" => "1. 1("
HEAD : period ◆ tablet ◆ ln ◆ lineNum
IDENTICAL: all 42170 items
=    : uruk-iii ◆ P006427 ◆ 6 ◆ 1
=    : uruk-iii ◆ P006427 ◆ 8 ◆ 1
=    : uruk-iii ◆ P006428 ◆ 16 ◆ 1
=    : uruk-iii ◆ P006428 ◆ 17 ◆ 2
=    : uruk-iii ◆ P006428 ◆ 19 ◆ 1
=    : uruk-iii ◆ P006428 ◆ 20 ◆ 2
=    : uruk-iii ◆ P006428 ◆ 22 ◆ 1
=    : uruk-iii ◆ P006428 ◆ 23 ◆ 2
=    : uruk-iii ◆ P006428 ◆ 24 ◆ 3
=    : uruk-iii ◆ P006428 ◆ 25 ◆ 4
=    : uruk-iii ◆ P006428 ◆ 26 ◆ 5
=    : uruk-iii ◆ P006428 ◆ 27 ◆ 6
=    : uruk-iii ◆ P006428 ◆ 28 ◆ 7
=    : uruk-iii ◆ P006428 ◆ 30 ◆ 1
=    : uruk-iii ◆ P006428 ◆ 31 ◆ 2
=    : uruk-iii ◆ P006428 ◆ 33 ◆ 1
=    : uruk-iii ◆ P448701 ◆ 41 ◆ 1
=    : uruk-iii ◆ P448701 ◆ 42 ◆ 2
=    : uruk-iii ◆ P448701 ◆ 44 ◆ 1
=    : uruk-iii ◆ P448701 ◆ 45 ◆ 2
=     and 42150 more
Number of results: TF 42170; GREP 42170


### Cross references

Cross references are the lines starting with `>>`.

Example:

```
>>P000014 oi2 
```

It means that the preceding line refers to tablet `P000014`, line `oi2`.
We do not interpret these values, but we have stored it in the feature *crossref*
as `P000014.oi2`.

If there are multiple crossrefs, we haven combined them in a comma-separated list.

In [27]:
def tfCrossrefs():
    crossrefs = []
    for tablet in F.otype.s('tablet'):
        tabletName = F.catalogId.v(tablet)
        period = F.period.v(tablet)
        for case in L.d(tablet, otype='case'):
            crossrefStr = F.crossref.v(case)
            if crossrefStr is None:
                continue
            theseCrossrefs = crossrefStr.split(',')
            fullNumber = F.fullNumber.v(case)
            ln = F.srcLnNum.v(case)
            origNumber = F.origNumber.v(case)
            theNumber = fullNumber if origNumber is None else origNumber
            for (offset, crossref) in enumerate(theseCrossrefs):
                crossrefs.append((period, tabletName, ln + 1 + offset, theNumber, crossref))
    return crossrefs

In [28]:
crossrefPat = '''
(?:
    >>
    \s*
    (\S+)
    (.*)
)
'''

In [29]:
def grepCrossrefs(gen):
    lineNumRe = re.compile(lineNumPat, re.X)
    crossrefRe = re.compile(crossrefPat, re.X)

    crossrefs = []
    curNum = None
    for (period, tablet, ln, line, skip) in gen:
        if skip:
            continue

        line = tweakLine(line)
            
        match = lineNumRe.match(line)
        if match:
            caseNum = match.group(1)
            caseNumClean = caseNum.replace('.', '').strip()
            curNum = caseNumClean
            continue
        match = crossrefRe.match(line)
        if match:
            cTablet = match.group(1)
            rest = match.group(2).strip()
            if rest:
                rest = ':'.join(rest.split(maxsplit=1))
            if rest:
                rest = f'.{rest}'
            crossrefs.append((period, tablet, ln, curNum, f'{cTablet}{rest}'))
        
    return crossrefs

If there are comments intervening a line and its crossrefs,
TF can no longer produce the exact line numbers for those crossrefs. They will be a bit of.
We will allow the comparison a bit of leeway in this respect.

In [30]:
COMP.checkSanity(
    ('lineNum', 'crossref'),
    grepCrossrefs,
    tfCrossrefs,
    leeway=1,
)

GREP: "1.1(" => "1. 1("
HEAD : period ◆ tablet ◆ ln ◆ lineNum ◆ crossref
IDENTICAL: all 6938 items
=    : uruk-iii ◆ P000456 ◆ 308 ◆ 1 ◆ P000014.oi2
=    : uruk-iii ◆ P000456 ◆ 311 ◆ 3 ◆ P000014.oiii5
=    : uruk-iii ◆ P000456 ◆ 317 ◆ 1 ◆ P000014.oii8
=    : uruk-iii ◆ P000456 ◆ 320 ◆ 3 ◆ P000014.ri2
=    : uruk-iii ◆ P002498 ◆ 6606 ◆ 1 ◆ Q000026.007
=    : uruk-iii ◆ P002498 ◆ 6608 ◆ 2 ◆ Q000026.008
=    : uruk-iii ◆ P002498 ◆ 6610 ◆ 3 ◆ Q000026.009
=    : uruk-iii ◆ P002498 ◆ 6612 ◆ 4 ◆ Q000026.010
=    : uruk-iii ◆ P002498 ◆ 6614 ◆ 5 ◆ Q000026.011
=    : uruk-iii ◆ P002498 ◆ 6618 ◆ 1 ◆ Q000026.n
=    : uruk-iii ◆ P002498 ◆ 6620 ◆ 2 ◆ Q000026.n
=    : uruk-iii ◆ P002498 ◆ 6622 ◆ 3 ◆ Q000026.n
=    : uruk-iii ◆ P002498 ◆ 6624 ◆ 4 ◆ Q000026.n
=    : uruk-iii ◆ P002498 ◆ 6628 ◆ 1 ◆ Q000026.colophon
=    : uruk-iii ◆ P464118 ◆ 16715 ◆ 1 ◆ Q000002.001
=    : uruk-iii ◆ P464118 ◆ 16717 ◆ 2 ◆ Q000002.002
=    : uruk-iii ◆ P464118 ◆ 16719 ◆ 3 ◆ Q000002.003
=    : uruk-iii ◆ P464118 ◆ 16721 ◆

## Comments

There are several types of comments: rulings (`$`), metadata (`#`), and object descriptions
(`@object`).

Comments can be associated with many kinds of source lines.

In [31]:
cTypes = set('''
    tablet
    face
    column
    line
    case
'''.strip().split())

### Object descriptions

The object descriptions are tied to tablets.

In [32]:
def tfObjects():
    tobjects = []
    for t in F.otype.s('tablet'):
        (tablet, column, line) = T.sectionFromNode(t)
        objects = (o for o in L.d(t, otype='comment') if F.type.v(o) == 'object')
        for obj in objects:
            tobjects.append((
                F.period.v(t), tablet, F.srcLnNum.v(obj), 
                F.catalogId.v(t), F.text.v(obj)
            ))
    return tobjects

In [33]:
objectDetect = '''
(?:
    ^@object
    \s+
    (.*)
)
'''

In [34]:
def grepObjects(gen):
    objectRe = re.compile(objectDetect, re.X)

    tobjects = []
    for (period, tablet, ln, line, skip) in gen:
        if skip:
            print(f'GREP: skipping duplicate tablet "{tablet}"')
            continue
        match = objectRe.match(line)
        if match:
            text = match.group(1).strip()
            tobjects.append((period, tablet, ln, tablet, text))
    return tobjects

In [35]:
COMP.checkSanity(
    ('text',),
    grepObjects,
    tfObjects,
)

GREP: skipping duplicate tablet "P002176"
GREP: skipping duplicate tablet "P252175"
HEAD : period ◆ tablet ◆ ln ◆ text
IDENTICAL: all 437 items
=    : uruk-iii ◆ P464118 ◆ 16712 ◆ P464118 ◆ composite text
=    : uruk-iii ◆ P471689 ◆ 23378 ◆ P471689 ◆ composite text
=    : uruk-iii ◆ P471682 ◆ 23996 ◆ P471682 ◆ composite text
=    : uruk-iii ◆ P471685 ◆ 24921 ◆ P471685 ◆ composite text
=    : uruk-iii ◆ P471683 ◆ 25676 ◆ P471683 ◆ composite text
=    : uruk-iii ◆ P471691 ◆ 25905 ◆ P471691 ◆ composite text
=    : uruk-iii ◆ P471694 ◆ 26157 ◆ P471694 ◆ composite text
=    : uruk-iii ◆ P471693 ◆ 27893 ◆ P471693 ◆ composite text
=    : uruk-iii ◆ P471688 ◆ 31515 ◆ P471688 ◆ composite text
=    : uruk-iii ◆ P471687 ◆ 33496 ◆ P471687 ◆ composite text
=    : uruk-iii ◆ P471692 ◆ 33939 ◆ P471692 ◆ composite text
=    : uruk-iii ◆ P471684 ◆ 36071 ◆ P471684 ◆ composite text
=    : uruk-iii ◆ P491489 ◆ 36902 ◆ P491489 ◆ composite text
=    : uruk-iii ◆ P471686 ◆ 40158 ◆ P471686 ◆ composite text
= 

### Rulings

These are not really comments, but markings on the tablets.

In order to find rulings that are associated to a node, we 
traverse from that node via `comments` edges.

In [36]:
def tfRulings():
    rulings = []
    for n in N():
        if F.otype.v(n) not in cTypes:
            continue
        cNodes = [c for c in E.comments.f(n) if F.type.v(c) == 'ruling']
        if not cNodes:
            continue
        t = L.u(n, otype='tablet')[0] if F.otype.v(n) != 'tablet' else n
        (tablet, column, line) = T.sectionFromNode(n)
        for cNode in cNodes:
            rulings.append((
                F.period.v(t), tablet, F.srcLnNum.v(cNode), 
                F.text.v(cNode)
            ))
    return rulings

In [37]:
rulingDetect = '''
(?:
    ^\$
    \s*
    (.*)
)
'''

In [38]:
def grepRulings(gen):
    rulingRe = re.compile(rulingDetect, re.X)

    rulings = []
    for (period, tablet, ln, line, skip) in gen:
        if skip:
            print(f'GREP: skipping duplicate tablet "{tablet}"')
            continue
        match = rulingRe.match(line)
        if match:
            text = match.group(1).strip()
            rulings.append((period, tablet, ln, text))
    return rulings

In [39]:
COMP.checkSanity(
    ('text',),
    grepRulings,
    tfRulings,
)

GREP: skipping duplicate tablet "P002176"
GREP: skipping duplicate tablet "P252175"
HEAD : period ◆ tablet ◆ ln ◆ text
IDENTICAL: all 4061 items
=    : uruk-iii ◆ P448701 ◆ 47 ◆ (not imaged)
=    : uruk-iii ◆ P448702 ◆ 55 ◆ beginning broken
=    : uruk-iii ◆ P448702 ◆ 61 ◆ beginning broken
=    : uruk-iii ◆ P448702 ◆ 62 ◆ blank space
=    : uruk-iii ◆ P448702 ◆ 65 ◆ beginning broken
=    : uruk-iii ◆ P448702 ◆ 66 ◆ blank space
=    : uruk-iii ◆ P448702 ◆ 68 ◆ (not imaged)
=    : uruk-iii ◆ P448703 ◆ 82 ◆ blank space
=    : uruk-iii ◆ P448703 ◆ 84 ◆ (not imaged)
=    : uruk-iii ◆ P471695 ◆ 99 ◆ n lines broken
=    : uruk-iii ◆ P471695 ◆ 102 ◆ rest broken
=    : uruk-iii ◆ P471695 ◆ 103 ◆ (for a total of 12 sub-cases with PNN)
=    : uruk-iii ◆ P471695 ◆ 107 ◆ blank space
=    : uruk-iii ◆ P471695 ◆ 108 ◆ rest broken
=    : uruk-iii ◆ P471695 ◆ 110 ◆ beginning broken
=    : uruk-iii ◆ P482082 ◆ 119 ◆ beginning broken
=    : uruk-iii ◆ P482082 ◆ 124 ◆ (not given)
=    : uruk-iii ◆ P482083

### Metadata

These are reall comments, in the sense that they do not correspond to 
information on the tablets, but remarks supplied by the transcribers.

As with rulings, in order to find metadat that is associated to a node, we 
traverse from that node via `comments` edges.

In [40]:
def tfMetas():
    metas = []
    for n in N():
        if F.otype.v(n) not in cTypes:
            continue
        cNodes = [c for c in E.comments.f(n) if F.type.v(c) == 'meta']
        if not cNodes:
            continue
        t = L.u(n, otype='tablet')[0] if F.otype.v(n) != 'tablet' else n
        (tablet, column, line) = T.sectionFromNode(n)
        for cNode in cNodes:
            metas.append((
                F.period.v(t), tablet, F.srcLnNum.v(cNode), 
                F.text.v(cNode)
            ))
    return metas

In [41]:
metaDetect = '''
(?:
    ^[#]
    \s*
    (.*)
)
'''

In [42]:
def grepMetas(gen):
    metaRe = re.compile(metaDetect, re.X)

    metas = []
    for (period, tablet, ln, line, skip) in gen:
        if skip:
            print(f'GREP: skipping duplicate tablet "{tablet}"')
            continue
        match = metaRe.match(line)
        if match:
            text = match.group(1).strip()
            metas.append((period, tablet, ln, text))
    return metas

In [43]:
COMP.checkSanity(
    ('text',),
    grepMetas,
    tfMetas,
)

GREP: skipping duplicate tablet "P002176"
GREP: skipping duplicate tablet "P252175"
HEAD : period ◆ tablet ◆ ln ◆ text
IDENTICAL: all 11308 items
=    : uruk-iii ◆ P006427 ◆ 2 ◆ version: 0.1
=    : uruk-iii ◆ P006427 ◆ 3 ◆ atf: lang qpc
=    : uruk-iii ◆ P006428 ◆ 12 ◆ version: 0.1
=    : uruk-iii ◆ P006428 ◆ 13 ◆ atf: lang qpc
=    : uruk-iii ◆ P448701 ◆ 37 ◆ atf: lang qpc
=    : uruk-iii ◆ P448702 ◆ 51 ◆ atf: lang qpc
=    : uruk-iii ◆ P448703 ◆ 72 ◆ atf: lang qpc
=    : uruk-iii ◆ P471695 ◆ 88 ◆ atf: lang qpc
=    : uruk-iii ◆ P482082 ◆ 115 ◆ atf: lang qpc
=    : uruk-iii ◆ P482083 ◆ 128 ◆ atf: lang qpc
=    : uruk-iii ◆ P482083 ◆ 137 ◆ calculation: 3(N01) ÷ 10 = 1(N39) 1(N24)
=    : uruk-iii ◆ P499393 ◆ 148 ◆ atf: lang qpc
=    : uruk-iii ◆ P504412 ◆ 167 ◆ atf: lang qpc
=    : uruk-iii ◆ P504413 ◆ 190 ◆ atf: lang qpc
=    : uruk-iii ◆ P006438 ◆ 200 ◆ atf: lang qpc
=    : uruk-iii ◆ P000014 ◆ 221 ◆ atf: lang qpc
=    : uruk-iii ◆ P000014 ◆ 227 ◆ = P000456 obv. ii 1
=    : uruk-iii ◆

## Graphemes

Note that we have loaded functions to produce atf representations
for signs, quads and clusters from the Cunei submodule of TF.
These functions take care of repeats, primes, variants modifiers, and flags.
See [additions](https://github.com/Dans-labs/text-fabric/wiki/Additions).

A complication is that there are missing line numbers in a few cases, 
so the usual grep pattern does not pick them up.

There a lines that start with `[` and with `|`, so we have to take care we get them.

There are also line numbers with a hyphen in it, such as `6-7`.

### Primes

First an overview of the occurrence of primes.

**N.B.:** This gathers primes on *signs*, *column* numbers and *case* numbers.

In [44]:
for (value, frequency) in F.prime.freqList():
    print(f'{frequency:>5} x {value}')

 5184 x 1


We also want so see the node types of primed entities.

In [45]:
primed = collections.Counter()
for n in F.prime.s(1):
    primed[F.otype.v(n)] += 1
for x in sorted(primed.items()):
    print(f'{x[1]:>5} x {x[0]}')

 4652 x case
  523 x column
    9 x sign


Now let us check the primes with grep, directly in the source files.
We look into lines starting with a (hierarchical number), followed by space,
and then later a single of double prime, but not one within a grapheme, such as `GA'AR`.

In [46]:
def tfPrimes():
    primes = []
    for n in F.prime.s(1):
        if F.otype.v(n) != 'sign':
            continue
        (tablet, column, line) = T.sectionFromNode(n)
        t = L.u(n, otype='tablet')[0]
        case = L.u(n, otype='case')[0]
        
        primes.append((F.period.v(t), tablet, F.srcLnNum.v(case), f"{CUNEI.atfFromSign(n)}"))
    return primes

In [47]:
primeDetect = f'''
(?:
    {lineNumDetect}
    (
        .*
        [\'"]
        [^A]
        .*
    )
)
'''

graphemePrimePat = '''
(?:
        (?:
            [0-9N]+
            \(
                [^)]+[\'"]
            \)
        )
    |
        (?:
            [A-Z0-9~@a-wyz\'-]+\'
        )
)
'''

In [48]:
def grepPrimes(gen):
    primes = []
    primeRe = re.compile(primeDetect, re.X)
    graphemeRe = re.compile(graphemePrimePat, re.X)
    for (period, tablet, ln, line, skip) in gen:
        if skip:
            continue
        line = tweakLine(line)
        match = primeRe.match(line)
        if match:
            material = match.group(1)
            if '"' in material:
                print(f'GREP: in "{material}": replacing " by \'')
                material = material.replace('"', "'")
            graphemes = graphemeRe.findall(material)
            for grapheme in graphemes:
                primes.append((period, tablet, ln, grapheme))
    return primes

In [49]:
COMP.checkSanity(
    ('grapheme',),
    grepPrimes,
    tfPrimes,
)

GREP: "1.1(" => "1. 1("
GREP: in "3(N41) 1(N24")# , [TAR~a] ": replacing " by '
HEAD : period ◆ tablet ◆ ln ◆ grapheme
IDENTICAL: all 9 items
=    : uruk-iii ◆ P411604 ◆ 48967 ◆ 1(N24')
=    : uruk-iii ◆ P411610 ◆ 49069 ◆ 1(N24')
=    : uruk-iii ◆ P411610 ◆ 49071 ◆ 1(N24')
=    : uruk-iii ◆ P411610 ◆ 49073 ◆ 1(N24')
=    : uruk-iii ◆ P411610 ◆ 49075 ◆ 1(N24')
=    : uruk-iii ◆ P411539 ◆ 49391 ◆ 1(N24')
=    : uruk-iii ◆ P006437 ◆ 54446 ◆ 1(N30c')
=    : uruk-iii ◆ P464140 ◆ 55938 ◆ 1(N24')
=    : uruk-iii ◆ P464140 ◆ 55939 ◆ 1(N24')
=     no more items
Number of results: TF 9; GREP 9


### Variants and modifiers

Overview of variants:

In [50]:
for (value, frequency) in F.variant.freqList():
    print(f'{frequency:>5} x {value}')

23804 x a
 4172 x b
 1532 x c
 1356 x a1
  703 x b1
  194 x a2
  187 x d
  127 x b2
   85 x f
   73 x a3
   40 x e
   29 x c2
   22 x c1
   22 x c3
   17 x v
   14 x c5
   13 x b3
   12 x a0
   12 x d1
   11 x c4
    6 x a4
    6 x g
    5 x d2
    4 x d4
    4 x h
    2 x 3a
    2 x d3
    1 x h2


Overview of modifiers outside a repeat expression, like `1(N57)@t`:

In [51]:
for (value, frequency) in F.modifier.freqList():
    print(f'{frequency:>5} x {value}')

  648 x g
  251 x t
   39 x n
    6 x r
    4 x s
    1 x c
    1 x v


Overview of modifiers within a repeat expression, like `7(N34@f)#`:

In [52]:
for (value, frequency) in F.modifierInner.freqList():
    print(f'{frequency:>5} x {value}')

   25 x f
   15 x t
    1 x r
    1 x v


So there are many variants and considerably fewer modifiers.

We look for variants and modifiers in the TF resource and by GREPping them from the sources.

In [53]:
def tfVarMod():
    varmods = []
    for n in F.otype.s('sign'):
        variant = F.variant.v(n)
        modifier = F.modifier.v(n)
        modifierInner = F.modifierInner.v(n)
        if variant is None and modifier is None and modifierInner is None:
            continue
        (tablet, column, line) = T.sectionFromNode(n)
        t = L.u(n, otype='tablet')[0]
        case = L.u(n, otype='case')[0]
        
        position = (F.period.v(t), tablet, F.srcLnNum.v(case))
        varmods.append((*position, f"{CUNEI.atfFromSign(n)}"))

        written = F.written.v(n)
        if written is not None:
            if '~' in written:
                varmods.append((*position, written))

    return varmods

#### Order
Modifiers and variants may come in any order.
The conversion has set *modifierFirst* on those items where the modifier precedes the variant.

Hence, when we fetch data from TF, we can and do put modifiers and variants in the right order.

Examples:

```
3. 1(N14) 8(N01) , RAD~a@g ERIM~a SZU2 A?
```

and cases with modifier and then variant:

```
4. 2(N01) , URUDU@g~b SZU2#
```

both from the same tablet P003407.

#### Uppercase
We encounter modifiers or variants in uppercase.
The conversion has brought them to lower case.
When we fetch data by grep, we perform this lowercasing before making the comparison.

#### Stray modifier
Somewhere in the source is `SUKUD@inversum`.
The conversion translates the `@inversum` to `@v`.
We have to mimick that when we do grep.

### Tweaks
During conversion, we found some problems in the sources and tweaked them.
When we grep, we must repeat those tweaks, in order to get comparable results.

See the [diagnostics](https://github.com/Dans-labs/Nino-cunei/blob/master/reports/diagnostics.tsv)

In [54]:
TWEAK_MATERIAL = (
#    ('4"', "4'"),
#    ('[,', ''),
    ('SA|L', 'SAL|'),
    ('~x(|EN~c.NUN~a|)', '~v (|EN~c.NUN~a|)a'),
    ('~x', '~v'),
    ('U2@~b', 'U2~b'),
#    (')|U', ') |U'),
    ('1N(02)', '1(N02)'),
    ('(1N', '1(N'),
#    ('~A', '~a'),
    ('{', '('),
    ('}', ')'),
#    ('sag-apin', 'sag-apin'),
#    ('@inversum', '@v'),
     (('KI@', -1), 'KI!'),
)

def tweakBeforeGrep(material):
    for (pat, rep) in TWEAK_MATERIAL:
        if type(pat) is tuple:
            (pat, pos) = pat
            if pos == 0:
                condition = material.startswith(pat)
                mark = ' (at start)'
            elif pos == -1:
                condition = material.endswith(pat)
                mark = ' (at end)'
        else:
            pos = None
            condition = pat in material
            mark = ''

        if condition:
            print(f'GREP: tweak "{pat}" => "{rep}"')
            if pos is None:
                material = material.replace(pat, rep)
            elif pos == 0:
                material = material.replace(pat, rep, 1)
            else:
                material = material[0:-len(pat)] + rep
    return material

In [55]:
upperPat = '''
(?:
    [~]([A-Z])
)
'''

In [56]:
upperRe = re.compile(upperPat, re.X)

def lower(match):
    return f'~{match.group(1).lower()}'

def graphemeTweaks(grapheme):
    if '@inversum' in grapheme:
        print(f'GREP: "@inversum" => "@v"')
        grapheme = grapheme.replace('@inversum', '@v')
    if '~a~a' in grapheme:
        print(f'GREP: "~a~a" => "~a"')
        grapheme = grapheme.replace('~a~a', '~a')
    if upperRe.search(grapheme):
        print(f'GREP: uppercase variant/modifier in grapheme changed to lowercase')
        grapheme = upperRe.sub(lower, grapheme)
    if '"' in grapheme:
        print(f'GREP: double prime replaced by single one')
        grapheme = grapheme.replace('"', "'")
    if '?#' in grapheme:
        print(f'GREP: flags order "?#" changed into "#?"')
        grapheme = grapheme.replace('?#', '#?')
    return grapheme

In [57]:
varmodDetect = f'''
(?:
    {lineNumDetect}
    (
        .*
        [@~]
        .*
    )
)
'''

graphemeVarmodPat = '''
(?:
        (?:
            [0-9N]+
            \(
                [^)]+
                [@~]
                [^)]+
            \)
        )
    |
        (?:
            [0-9N]+
            \(
                [^)]+
            \)
            @[a-z]
        )
    |
        (?:
            [A-Z0-9a-wyz\'-]+
            [@~]
            [0-9~@a-wyzA-WYZ]+
        )
)
'''

In [58]:
def grepVarMod(gen):
    varmodRe = re.compile(varmodDetect, re.X)
    graphemeRe = re.compile(graphemeVarmodPat, re.X)

    varmods = []
    for (period, tablet, ln, line, skip) in gen:
        if skip:
            continue
        line = tweakLine(line)
        line = tweakBeforeGrep(line.strip())
        match = varmodRe.match(line)
        if match:
            material = match.group(1)
            graphemes = graphemeRe.findall(material)
            for grapheme in graphemes:
                grapheme = graphemeTweaks(grapheme)
                varmods.append((period, tablet, ln, grapheme))
    return varmods

In [59]:
COMP.checkSanity(
    ('grapheme',),
    grepVarMod,
    tfVarMod,
)

GREP: "@inversum" => "@v"
GREP: tweak "SA|L" => "SAL|"
GREP: tweak "1N(02)" => "1(N02)"
GREP: tweak "{" => "("
GREP: tweak "}" => ")"
GREP: tweak "~x" => "~v"
GREP: tweak "~x" => "~v"
GREP: uppercase variant/modifier in grapheme changed to lowercase
GREP: tweak "KI@" => "KI!"
GREP: tweak "(1N" => "1(N"
GREP: tweak "~x(|EN~c.NUN~a|)" => "~v (|EN~c.NUN~a|)a"
GREP: tweak "~x" => "~v"
GREP: tweak "~x" => "~v"
GREP: "1.1(" => "1. 1("
GREP: "~a~a" => "~a"
GREP: "~a~a" => "~a"
GREP: "~a~a" => "~a"
GREP: "~a~a" => "~a"
GREP: "~a~a" => "~a"
GREP: "~a~a" => "~a"
GREP: tweak "U2@~b" => "U2~b"
GREP: uppercase variant/modifier in grapheme changed to lowercase
GREP: uppercase variant/modifier in grapheme changed to lowercase
GREP: uppercase variant/modifier in grapheme changed to lowercase
HEAD : period ◆ tablet ◆ ln ◆ grapheme
IDENTICAL: all 32807 items
=    : uruk-iii ◆ P006427 ◆ 8 ◆ SANGA~a
=    : uruk-iii ◆ P006428 ◆ 26 ◆ DUG~b
=    : uruk-iii ◆ P448701 ◆ 42 ◆ AB~a
=    : uruk-iii ◆ P448701 ◆ 42

### Flags

We have several features for flags: 

mark | feature | comments
----|---
`*`|*collation* | not encountered in uruk iii-iv
`#`|*damage*
`?`|*uncertain*
`!`|*remarkable*
`!(`ggg`)`|*written*

#### A bit of research
We start by surveying the possible values, including on which node types they occur

In [60]:
flagFeatures = '''
    damage
    remarkable
    written
    uncertain
'''.strip().split()

In [61]:
flagNodeOverview = collections.Counter()
flagNodeTypes = set()

for n in N():
    for ft in flagFeatures:
        value = Fs(ft).v(n)
        if not value: continue
        nType = F.otype.v(n)
        flagNodeTypes.add(nType)
        flagNodeOverview[f'{nType}-{ft}-{value}'] += 1
for (combi, amount) in sorted(flagNodeOverview.items(), key=lambda x: (-x[1], x[0])):
    print(f'{amount:>6} x {combi}')

 19945 x sign-damage-1
  3727 x sign-uncertain-1
  1045 x quad-damage-1
   321 x quad-uncertain-1
    11 x sign-remarkable-1
     2 x sign-written-KASKAL
     1 x sign-written-GURUSZ~a
     1 x sign-written-IB~a


Let us see whether there are any cooccurrences of flags.

In [62]:
flagCombis = collections.Counter()

for n in N():
    if F.otype.v(n) not in flagNodeTypes:
        continue
    values = []
    for ft in flagFeatures:
        rawValue = Fs(ft).v(n)
        value = f'{"*":^10}' if rawValue is None else f'{ft:^10}' if rawValue else f'{"":^10}'
        values.append(value)

    combi = '-'.join(values)
    flagCombis[combi] += 1

for (combi, amount) in sorted(flagCombis.items(), key=lambda x: (-x[1], x[0])):
    print(f'{amount:>6} x {combi}')

128414 x     *     -    *     -    *     -    *     
 18385 x   damage  -    *     -    *     -    *     
  2605 x   damage  -    *     -    *     -uncertain 
  1442 x     *     -    *     -    *     -uncertain 
    11 x     *     -remarkable-    *     -    *     
     3 x     *     -    *     - written  -    *     
     1 x     *     -    *     - written  -uncertain 


We need to address the question about order of flags.

A quick inspection in the corpus yields:

* damage-uncertain (`#?`) and uncertain-damage (`?#`), but the latter is very rare and all cases
  occur in the diagnostics;
* uncertain-remarkable (`?!`) does not occur, and remarkable-written-uncertain (`!(`ggg`)?` does occur.

Based on this observation, and assuming that the order between *damage* and *uncertain* is not relevant,
we produce flags always in the order:

* *damage* *remarkable* *written* *uncertain*

When grepping, we have to normalize `?#` to `#?`.

There is one weird case, diagnosed by the conversion, in tablet P471687:

```
C1. |DUGb+?|
```

Here we see an operator `+` of which the second operand `?` consists of a flag without a grapheme.
This occurs only once, so I think it is a mistake.

Yet we'll find it when we search with TF, so we exclude it.

In [63]:
def tfFlags():
    flags = []
    for n in F.otype.s('sign'):
        values = [Fs(ft).v(n) for ft in flagFeatures]
        if all(value is None for value in values):
            continue
        (tablet, column, line) = T.sectionFromNode(n)
        t = L.u(n, otype='tablet')[0]
        case = L.u(n, otype='case')[0]
        
        sign = f"{CUNEI.atfFromSign(n, flags=True)}"
        if sign == '?':
            print(f'TF: grapheme-less flag skipped in "{line}"')
            continue
        flags.append((F.period.v(t), tablet, F.srcLnNum.v(case), f"{CUNEI.atfFromSign(n, flags=True)}"))
    return flags

In [64]:
flagDetect = f'''
(?:
    {lineNumDetect}
    (
        .*
        [!?#]
        .*
    )
)
'''

flagsPat = '''
(?:
        (?:
            !\([^)]*\)
        )
    |
        [!#?]
)
'''

graphemeFlagPat = f'''
(?:
        (?:
            [0-9N]+
            \(
                [^)]+
            \)
            {flagsPat}+
        )
    |
        (?:
            [A-Z0-9~@a-wyz\'-]+
            {flagsPat}+
        )
)
'''

In [65]:
def grepFlags(gen):
    flagsRe = re.compile(flagDetect, re.X)
    graphemePat = re.compile(graphemeFlagPat, re.X)

    flags = []

    for (period, tablet, ln, line, skip) in gen:
        if skip:
            continue
        line = tweakLine(line)
        line = tweakBeforeGrep(line.strip())

        match = flagsRe.match(line)
        if match:
            material = match.group(1)
            graphemes = graphemePat.findall(material)
            for grapheme in graphemes:
                grapheme = graphemeTweaks(grapheme)
                flags.append((period, tablet, ln, grapheme))
    return flags

In [66]:
COMP.checkSanity(
    ('grapheme',),
    grepFlags,
    tfFlags,
)

TF: grapheme-less flag skipped in "C"
GREP: "@inversum" => "@v"
GREP: flags order "?#" changed into "#?"
GREP: flags order "?#" changed into "#?"
GREP: flags order "?#" changed into "#?"
GREP: tweak "SA|L" => "SAL|"
GREP: tweak "1N(02)" => "1(N02)"
GREP: tweak "{" => "("
GREP: tweak "}" => ")"
GREP: tweak "~x" => "~v"
GREP: tweak "~x" => "~v"
GREP: uppercase variant/modifier in grapheme changed to lowercase
GREP: tweak "KI@" => "KI!"
GREP: tweak "(1N" => "1(N"
GREP: tweak "~x(|EN~c.NUN~a|)" => "~v (|EN~c.NUN~a|)a"
GREP: tweak "~x" => "~v"
GREP: tweak "~x" => "~v"
GREP: "1.1(" => "1. 1("
GREP: double prime replaced by single one
GREP: "~a~a" => "~a"
GREP: tweak "U2@~b" => "U2~b"
HEAD : period ◆ tablet ◆ ln ◆ grapheme
IDENTICAL: all 21265 items
=    : uruk-iii ◆ P006427 ◆ 8 ◆ SANGA~a?
=    : uruk-iii ◆ P448702 ◆ 58 ◆ KASZ~a?
=    : uruk-iii ◆ P448702 ◆ 59 ◆ 6(N14)#?
=    : uruk-iii ◆ P448702 ◆ 63 ◆ SUKUD@v?
=    : uruk-iii ◆ P471695 ◆ 105 ◆ ISZ~a#?
=    : uruk-iii ◆ P471695 ◆ 111 ◆ 6(N01

### All signs

Now it is time to do a rigorous comparison of all signs in the transcriptions and in TF.

Up till now we included only signs that had primes, variants, modifiers or flags attached to them.
Now we extend the comparison to all of them.

We still ignore the quad structures with operators and the bracketing of quads (clusters).
But we do draw their component signs into the comparison.

This comparison lists all signs in textual order, in TF and in GREP, and compares the two lists.

In [67]:
def tfSigns():
    signs = []
    for n in F.otype.s('sign'):
        if F.grapheme.v(n) == '':
            continue
        (tablet, column, line) = T.sectionFromNode(n)
        t = L.u(n, otype='tablet')[0]
        case = L.u(n, otype='case')[0]
        
        sign = f"{CUNEI.atfFromSign(n, flags=True)}"
        if sign == '?':
            print(f'TF: grapheme-less flag skipped in "{line}"')
            continue
        signs.append((F.period.v(t), tablet, F.srcLnNum.v(case), sign))
    return signs

The regular expression that greps all graphemes from the transcription is quite daunting.
Comapare this with the relative easy by which you get all this from the text-fabric representation.

In [68]:
smallPat = '''
(?:
    [A-Zn]
)
'''

signDetect = f'''
(?:
    {lineNumDetect}
    (.*)
)
'''

flagsModVarPat = '''
(?:
        (?:
            !\([^)]*\)
        )
    |
        [!#?]
    |
        (?:
            @[A-WYZa-wyz]+
        )
    |
        (?:
            ~[A-WYZa-wyz0-9]+
        )
)
'''

graphemePat = f'''
(?:
        (?:
            \.\.\.                        # three dots, mostly in [...]
        )
    |
        (?:
            [0-9N]+\([^)]+\)               # a repeat, e.g. 5(N024)
            {flagsModVarPat}*
        )
    |
        (?:
            [A-Z0-9a-wyzû\'-]{{2,}}       # a plain grapheme, e.g. GA'AR
            {flagsModVarPat}*
        )
    |
        (?:
            {smallPat}                    # a single letter grapheme: X or n, mostly in [n]
            {flagsModVarPat}*
        )
)
'''

In [69]:
def grepSigns(gen):
    signRe = re.compile(signDetect, re.X)
    graphemeRe = re.compile(graphemePat, re.X)

    signs = []
    for (period, tablet, ln, line, skip) in gen:
        if skip:
            continue
        line = tweakLine(line)
        match = signRe.match(line)
        if match:
            material = match.group(1).strip()
            material = tweakBeforeGrep(material)
            graphemes = graphemeRe.findall(material)
            for grapheme in graphemes:
                grapheme = graphemeTweaks(grapheme)
                signs.append((period, tablet, ln, grapheme))
    return signs

The next one will take 5-10 seconds.

In [70]:
COMP.checkSanity(
    ('grapheme',),
    grepSigns,
    tfSigns,
)

GREP: "@inversum" => "@v"
GREP: flags order "?#" changed into "#?"
GREP: flags order "?#" changed into "#?"
GREP: flags order "?#" changed into "#?"
GREP: tweak "SA|L" => "SAL|"
GREP: tweak "1N(02)" => "1(N02)"
GREP: tweak "{" => "("
GREP: tweak "}" => ")"
GREP: tweak "~x" => "~v"
GREP: tweak "~x" => "~v"
GREP: uppercase variant/modifier in grapheme changed to lowercase
GREP: tweak "KI@" => "KI!"
GREP: tweak "(1N" => "1(N"
GREP: tweak "~x(|EN~c.NUN~a|)" => "~v (|EN~c.NUN~a|)a"
GREP: tweak "~x" => "~v"
GREP: tweak "~x" => "~v"
GREP: "1.1(" => "1. 1("
GREP: double prime replaced by single one
GREP: "~a~a" => "~a"
GREP: "~a~a" => "~a"
GREP: "~a~a" => "~a"
GREP: "~a~a" => "~a"
GREP: "~a~a" => "~a"
GREP: "~a~a" => "~a"
GREP: tweak "U2@~b" => "U2~b"
GREP: uppercase variant/modifier in grapheme changed to lowercase
GREP: uppercase variant/modifier in grapheme changed to lowercase
GREP: uppercase variant/modifier in grapheme changed to lowercase
HEAD : period ◆ tablet ◆ ln ◆ grapheme
IDENTICAL

## Quads

Quads are the compositions of signs by operators. Operators can be applied several times,
so quads themselves can be the building blocks for other quads.

In transcription, the outermost quads are what you get if you split the transcription
line (the part after the line number) of white space.

Normally, if an outer quad is complex, it is written between `| |`.

Quads and sub-quads may be augmented with variants, modifiers and flags.

### Outer complex quads

Let us first check whether we see the same outer quads by TF as by GREP.
We are not interested in the simple quads.
They are the signs, and we have already checked them.
So we look for all outer complex quads, i.e. quads with an operator in them,
such as `x` or `+` or `.`

In the transcriptions, complex outer quads should be surrounded by `| |`.
However, in this corpus there are a few cases where one or both of the surrounding
`|` are missing.

We make an explicit list of these cases, and will correct the GREP results for these.

In TF, outer quads are characterized by not having an incoming `sub` edge from an other quad.
Remember that edges connect the complex quads with their component quads.

**N.B.:** There are also `sub` edges between clusters and the quads that they contain,
so when we check for incoming `sub` edges, we must only count the ones coming from a quad.

In TF we recognize a complex quad as one having an outgoing `sub` edge.

In [71]:
def tfQuads():
    quads = []
    for n in F.otype.s('quad'):
        if any(
            F.otype.v(parent) == 'quad' for parent in E.sub.t(n)
        ) or not E.sub.f(n):
            continue
        (tablet, column, line) = T.sectionFromNode(n)
        t = L.u(n, otype='tablet')[0]
        case = L.u(n, otype='case')[0]
        
        quad = f"{CUNEI.atfFromQuad(n, flags=True)}"
        quads.append((F.period.v(t), tablet, F.srcLnNum.v(case), quad))
    return quads

### Variants: inside or outside?

It seems that in the transcriptions there are two ways to augment an outer quad with a variant:

`|(QQxRR)~a|` versus `|QQxRR|~a`

Both occur

In P000783 we have
```
6. 2(N14) , |(SZAxHI@g~a)~b|
```

whereas in P252180 we have

```
4. UB URI3~a |BAD+DISZ|~a EN~a
```

### Variants: extra level of brackets?

What binds stronger: an operator, or a variant?
Consider the following line of P006326:

```
1.b2. 2(N14)# 3(N01)# , |GA2~a1x(SUKUD&SUKUD)~b|#
```

If variants bind stronger than operators, we must read the big quad as

```
|GA2~a1x((SUKUD&SUKUD)~b)|#
```

If operators bind stronger, we need to read it as

```
|(GA2~a1x(SUKUD&SUKUD))~b|#
```

A variant is unary operator, whereas an operator is a binary operator,
so it makes much more sense to opt for the first interpretation.

However, in the corpus we see cases where there are unnecessary brackets
for variants. See the following line in P002269:

```
1.a. 1(N01) , |NINDA2x((UDU~a+TAR)~b)| KU6~a
```

Here the brackets around `UDU~a+TAR)~b` are unneccesary. 
If variants bind stronger, then this should have the same meaning:

```
1.a. 1(N01) , |NINDA2x(UDU~a+TAR)~b| KU6~a
```

Since both bracketings occur, and since TF stores the abstract structure, and not
the surface bracketing, there is no way for TF to remember the bracketing.

We could add a feature to remember this, but as the meaning is not changed by extra
brackets, it is silly to do so.

Hence, after GREPPING, we will remove these kinds of brackets.

### More bracket issues

In P005112 we find a line

```
1. [...] , |U4x2(N01).(2(N14).1(N08))| EN~a PA~a
```

The quad seems the composition of three subquads:
`U4` and `2(N01)` and `2(N14).1(N08)`.

But whereas it has been made explicit in which order the `.` operator should be applied,
we do not know that for `x` versus `.`

There are three possible meanings of `GxH.J`

1. `Gx(H.J)`
2. `(GxH).J`
3. the order is immaterial, the previous meanings are equal

Given the fact that there are no brackets, I opt for the third possibility.

### The most complex quads

Here are a few really intricate quads:

In P005573, column 2, line 2b2

P005381 2 1

Let's retrieve them using with a bit of Text-Fabric agility:

```
2.b2. , (|(HIx1(N57))&(HI+1(N57))| EN~a)a 
```

In [72]:
for passage in (
    ('P005573', 'obverse:2', '2', '2b2'),
    ('P005381', 'obverse:2', '1'),
):
    line = T.nodeFromSection(passage[0:3])
    cases = L.d(line, otype='case')
    caseNr = passage[3] if len(passage) >= 4 else None
    for case in cases:
        if not caseNr or F.fullNumber.v(case) == caseNr:
            print(F.srcLn.v(case))

2.b2. , (|(HIx1(N57))&(HI+1(N57))| EN~a)a 
1. 3(N04) , |GISZ.TE| GAR |SZU2.((HI+1(N57))+(HI+1(N57)))| GI4~a


In [73]:
quadVarPat = '''
(?:
    \|
        ([^|]+)
    \|
    ~
        ([a-wyz0-9A-WYZ]+)
)
'''

quadVarRe = re.compile(quadVarPat, re.X)

def quadVarReplace(match):
    quad = match.group(1)
    var = match.group(2)
    print(f'GREP: pulling variant inside quad: "|{quad}|~{var}" => |({quad})~{var}|')
    return f'|({quad})~{var}|'

In [74]:
quadBracketPat = '''
(?:
    \(
        \(
            ([^() ]+)
        \)
        ~
        ([a-wyz0-9A-WYZ]+)
    \)
)
'''

quadBracketRe = re.compile(quadBracketPat, re.X)

def quadBracketReplace(match):
    quad = match.group(1)
    var = match.group(2)
    print(f'GREP: removing brackets around a variant: "(({quad})~{var})" => ({quad})~{var}')
    return f'({quad})~{var}'

In [75]:
operatorPat = '''
(?:
    [x%&.:+]
)
'''

In [76]:
MISSING_PIPES = {
       ('uruk-iii', 16730): '|SZE~a+NAM2|',
       ('uruk-iii', 16896): '|EN~a+NUN~a| UTUL~a',
       ('uruk-iii', 16962): 'GAL~a |EZEN~b+6N57|',
       ('uruk-iii', 25956): '|GISZ+SZU2~a| |SZE~a+SZE~a|',
       ('uruk-iii', 26281): '|GISZ.tenû| E2~b',
       ('uruk-iii', 31523): '|MUD3.gunû|',
       ('uruk-iii', 40186): '|SZUBUR+1(N57)|',
       ('uruk-iii', 40188): '|SZUBUR+2(N57)|',
       ('uruk-iii', 45102): '|ZATU737xDI| SANGA~a',
       ('uruk-iii', 45533): '|SZE~a+NAM2| A',
       ('uruk-iii', 52806): '1(N01) , UDUNITA~a |IDIN+1(N57)|',
       ('uruk-iii', 55110): '|U4+1(N08)| |GI+A#|',
}

In [77]:
complexQuadDetect = f'''
(?:
    {lineNumDetect}
    (
        .*
        \S
        {operatorPat}
        \S
        .*
    )
)
'''

complexQuadPat = f'''
(?:
    \|
        \S+
        {operatorPat}
        \S+
    \|
    {flagsModVarPat}*
)
'''

In [78]:
quadRe = re.compile(complexQuadPat, re.X)

In [79]:
def grepQuads(gen):
    quadsRe = re.compile(complexQuadDetect, re.X)
    quadRe = re.compile(complexQuadPat, re.X)

    quads = []
    for (period, tablet, ln, line, skip) in gen:
        if skip:
            continue
        line = tweakLine(line)
        material = MISSING_PIPES.get((period, ln), None)
        if material == None:
            match = quadsRe.match(line)
            if match:
                material = match.group(1).strip()
                material = tweakBeforeGrep(material)
        if material is not None:
            for quad in quadRe.findall(material):
                quad = graphemeTweaks(quad)
                quad = quadVarRe.sub(quadVarReplace, quad)
                quad = quadBracketRe.sub(quadBracketReplace, quad)
                quads.append((period, tablet, ln, quad))
    return quads

In [80]:
COMP.checkSanity(
    ('quad',),
    grepQuads,
    tfQuads,
)

GREP: tweak "SA|L" => "SAL|"
GREP: tweak "{" => "("
GREP: tweak "}" => ")"
GREP: tweak "~x(|EN~c.NUN~a|)" => "~v (|EN~c.NUN~a|)a"
GREP: "1.1(" => "1. 1("
GREP: removing brackets around a variant: "((UDU~a+TAR)~b)" => (UDU~a+TAR)~b
GREP: removing brackets around a variant: "((UDU~axTAR)~a)" => (UDU~axTAR)~a
GREP: pulling variant inside quad: "|BAD+DISZ|~a" => |(BAD+DISZ)~a|
HEAD : period ◆ tablet ◆ ln ◆ quad
IDENTICAL: all 3733 items
=    : uruk-iii ◆ P006428 ◆ 26 ◆ |DUG~bx1(N57)|
=    : uruk-iii ◆ P448702 ◆ 63 ◆ |U4x1(N01)|
=    : uruk-iii ◆ P448703 ◆ 76 ◆ |U4.1(N08)|
=    : uruk-iii ◆ P448703 ◆ 77 ◆ |U4.1(N08)|
=    : uruk-iii ◆ P448703 ◆ 78 ◆ |U4.1(N08)|#
=    : uruk-iii ◆ P448703 ◆ 78 ◆ |GI&GI|#
=    : uruk-iii ◆ P448703 ◆ 79 ◆ |U4.1(N08)|#
=    : uruk-iii ◆ P448703 ◆ 80 ◆ |U4.1(N08)|
=    : uruk-iii ◆ P482083 ◆ 135 ◆ |U4x3(N01)|
=    : uruk-iii ◆ P499393 ◆ 153 ◆ |LAGAB~bxX|
=    : uruk-iii ◆ P499393 ◆ 155 ◆ |MUSZEN.X|
=    : uruk-iii ◆ P504412 ◆ 177 ◆ |SILA3~dxNI~a|
=    : uruk-iii

## Clusters

Clusters are groupings of outermost quads.
The transcription uses a variety of brackets for several kinds of clustering.

In our corpus we encounter:

* `[ ]` uncertain
* `( )` proper names
* `< >` supplied material

Clusters may be nested.

Most clusters are trivial: `[...]`.

In [81]:
def tfClusters():
    clusters = []
    for n in F.otype.s('cluster'):
        kind = F.kind.v(n)
        (tablet, column, line) = T.sectionFromNode(n)
        t = L.u(n, otype='tablet')[0]
        case = L.u(n, otype='case')[0]
        
        cluster = f"{CUNEI.atfFromCluster(n)}"
        clusters.append((F.period.v(t), tablet, F.srcLnNum.v(case), cluster))
    return clusters

In [82]:
clusterOpen = '[(\[<]'
clusterClose = '[)\]>]'
openCluster = set('[(<')

clusterDetect = f'''
(?:
    {lineNumDetect}
    (
        (?:
                {clusterOpen}
            |
                (?:
                    .*\s
                    {clusterOpen}
                )
        )
        .*
    )
)
'''

nextMaterialPat = f'''
(?:
    \s{clusterOpen}
)
'''

clusterPat = f'''
(?:
    {clusterOpen}
    (?:
        (?:
                {complexQuadPat}
            |
                {graphemePat}
        )
    )
    (?:
        \s
        (?:
                {complexQuadPat}
            |
                {graphemePat}
        )
    )*
    {clusterClose}
    a?
)
'''

cluster2Pat = f'''
(?:
    {clusterOpen}
    (?:
        (?:
                {clusterPat}
            |
                {complexQuadPat}
            |
                {graphemePat}
        )
    )
    (?:
        \s
        (?:
                {clusterPat}
            |
                {complexQuadPat}
            |
                {graphemePat}
        )
    )*
    {clusterClose}
    a?
)
'''

In [83]:
clustersRe = re.compile(clusterDetect, re.X)
clusterRe = re.compile(cluster2Pat, re.X)
nextRe = re.compile(nextMaterialPat, re.X)

def getOuterClusters(material):
    clusters = []
    rest = 0
    while rest < len(material):        
        if material[rest] in openCluster:
            clusterMatch = clusterRe.match(material[rest:])
            if clusterMatch:
                clusterRaw = clusterMatch.group(0)
                clusters.append(clusterRaw)
                clusters.extend(getOuterClusters(clusterRaw[1:-1]))
                rest += len(clusterRaw)
            else:
                rest += 1
        else:
            nextMatch = nextRe.search(material[rest:])
            if nextMatch:
                rest += nextMatch.start() + 1
            else:
                break
    return clusters

def grepClusters(gen):
    clusters = []
    for (period, tablet, ln, line, skip) in gen:
        if skip:
            continue
        line = tweakLine(line)
        line = tweakBeforeGrep(line)
        line = line.replace(' , ', ' ').replace(',', '')

        match = clustersRe.match(line)
        if match:
            material = match.group(1).strip()
            clustersRaw = getOuterClusters(material)
            for clusterRaw in clustersRaw:
                cluster = graphemeTweaks(clusterRaw)
                clusters.append((period, tablet, ln, cluster))
    return clusters

In [84]:
COMP.checkSanity(
    ('cluster',),
    grepClusters,
    tfClusters,
)

GREP: tweak "SA|L" => "SAL|"
GREP: tweak "1N(02)" => "1(N02)"
GREP: tweak "{" => "("
GREP: tweak "}" => ")"
GREP: tweak "~x" => "~v"
GREP: tweak "~x" => "~v"
GREP: tweak "(1N" => "1(N"
GREP: tweak "~x(|EN~c.NUN~a|)" => "~v (|EN~c.NUN~a|)a"
GREP: tweak "~x" => "~v"
GREP: tweak "~x" => "~v"
GREP: "1.1(" => "1. 1("
GREP: tweak "U2@~b" => "U2~b"
HEAD : period ◆ tablet ◆ ln ◆ cluster
IDENTICAL: all 32959 items
=    : uruk-iii ◆ P006427 ◆ 6 ◆ [...]
=    : uruk-iii ◆ P006427 ◆ 8 ◆ [...]
=    : uruk-iii ◆ P006428 ◆ 33 ◆ [...]
=    : uruk-iii ◆ P448702 ◆ 56 ◆ [n]
=    : uruk-iii ◆ P448702 ◆ 56 ◆ [...]
=    : uruk-iii ◆ P448702 ◆ 57 ◆ [n]
=    : uruk-iii ◆ P448702 ◆ 59 ◆ [n]
=    : uruk-iii ◆ P448702 ◆ 59 ◆ [...]
=    : uruk-iii ◆ P471695 ◆ 93 ◆ (EN~a DU ZATU759)a
=    : uruk-iii ◆ P471695 ◆ 94 ◆ (BAN~b KASZ~c)a
=    : uruk-iii ◆ P471695 ◆ 95 ◆ (KI@n SAG)a
=    : uruk-iii ◆ P471695 ◆ 96 ◆ [...]
=    : uruk-iii ◆ P471695 ◆ 97 ◆ (3(N57) PAP~a)a
=    : uruk-iii ◆ P471695 ◆ 98 ◆ (SZU KI X)a
=    : u

## Conclusion

Here ends the checking.

This notebook has tested all patterns and quantities found in the transcriptions.

By a somewhat convoluted GREP we have extracted patterns from the sources.

By somewhat contrived TF alchemy we have produced the same patterns from the Text-Fabric
representation of the sources.

Then we have made a rigorous comparison: we have checked wether both methods found exactly
the same sequence of values.

And yes, it is so.